In [4]:
%load_ext autoreload
import sys
sys.path.append("..")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Helium

In [5]:
%autoreload

import numpy as np

import darkhistory.physics as phys

## TLA with Helium

The TLA with helium looks like the following set of equations (from astro-ph/9909275):

$$ \frac{dx_\text{p}}{dz} = \mathcal{C}_\text{H} \left(x_\text{e} x_\text{p} n_\text{H} \alpha_\text{H} - \beta_\text{H} (1 - x_p) e^{- \nu_{\text{H} 2s} / T_M} \right)$$

$$ \frac{dx_\text{HeII}}{dz} = \mathcal{C}_\text{He} \left( 
    x_e x_\text{HeII} n_\text{H} \alpha_\text{HeI} - \beta_\text{HeI} \left( f_\text{He} - x_\text{HeII} \right) e^{- \nu_{\text{HeI 2}^1s} / T_M} \right) $$
    
where

$$\mathcal{C}_\text{H} = \frac{1 + K_\text{H} \Lambda_\text{H} n_\text{H} (1 - x_p)}{H(z) (1+z) \left( 1 + K_\text{H} (\Lambda_\text{H} + \beta_\text{H}) n_\text{H} (1 - x_p) \right)} $$

and similarly

$$\mathcal{C}_\text{He} = \frac{1 + K_\text{HeI} \Lambda_\text{He} n_\text{H} \left(f_\text{He} - x_\text{HeII} \right) e^{-\nu_\text{ps}/T_M}}{H(z) (1+z) \left( 1 + K_\text{HeI} (\Lambda_\text{He} + \beta_\text{HeI}) n_\text{H} (f_\text{He} - x_\text{HeII}) e^{- \nu_\text{ps}/T_M} \right)} $$

$\mathcal{C}_\text{H}$ should be the Peebles-C factor that we already have (we will check this numerically below), while $\mathcal{C}_\text{He}$ is the equivalent factor for helium.

Let's try to obtain the solution to these equations, together with the usual temperature evolution equations.

In [32]:
def C_H(rs, xp, T_m):
    lambda_H_2p = 2*np.pi*phys.hbar*phys.c/phys.lya_eng
    K_H = lambda_H_2p**3/(8*np.pi*phys.hubble(rs))
    Lambda_H = 8.22458
    
    numer = 1 + K_H*Lambda_H*phys.nH*rs**3*(1 - xp)
    denom = numer + K_H*4*phys.beta_ion(phys.TCMB(rs))*phys.nH*rs**3
    
    return numer/denom/(phys.hubble(rs)*rs)


def dxp_dz(xp, xe, rs, T_m):
    
    return C_H(xp, rs) * (
        xe*xp*phys.nH*rs**3*alpha_H 
        - beta_H(1 - xp)*np.exp(-nu_H_2s/T_m)
    )

In [35]:
rs = 7000
xp = 0.5
T_m = 0.003

print(C_H(rs, xp, T_m))
print(phys.peebles_C(xp, rs)/phys.hubble(rs)/rs)

3.8988609133958407
3.90127920708969
